In [1]:
import os
import math
import numpy as np
import pandas as pd
import ccxt
import talib
from datetime import datetime
import tensorflow as tf
from tensorflow.python.keras.utils.np_utils import to_categorical
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from script.fetch_history_data import binance_fetch_history_price, binance_single_fetch_history_price
from script.preprocess import *
from sklearn.preprocessing import StandardScaler
from script.transformer_timestep import *
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import pickle
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

Tensorflow version: 2.6.5


In [3]:
def calculate_z_score_input_format(df_cal):
    for col in zscore_col:
        df_cal[col] = (df_cal[col] - zscore_mean) / zscore_std
    return df_cal[input_col], df_cal

def restore_single_zscore_close_price(pred_single_arr, all_col_df, target_col='pred_Close_MA_ptc_10', moving_avg_step=10, seq_len=128):
    # copy close, close_MA_10, to pred_df
    pred_close_MA_ptc_zscore = pred_single_arr[0][0]
    input_df_all_columns = all_col_df.reset_index(drop=True).copy()
    # restore z_score
    pred_close_MA_ptc = pred_close_MA_ptc_zscore*zscore_std + zscore_mean
    previois_avg      = input_df_all_columns['Close_MA_10'][seq_len-1]
    close_avg         = previois_avg * (pred_close_MA_ptc + 1)
    restored_close    = (close_avg * moving_avg_step) - (input_df_all_columns.loc[seq_len-(moving_avg_step -1):seq_len- 1, 'Close'].sum())
    return restored_close

def restore_single_zscore_close_price_v3(test_pred, df_zscore_total, seq_len=128):
    single_df = df_zscore_total.reset_index(drop=True).copy()
    # df_zscore_output['pred_Close']='-'
    # # 128~len(df)
    # for i in range(seq_len, len(df_zscore_output)):
    #     # restore the model predicted_ptc
    pred_ptc = test_pred[0][0]*zscore_std + zscore_mean
    # choose the single_df[0:128] (index 0~127) 
    # take the final MA_10
    pre_MA = single_df['Close_MA_10'][single_df.index[-1]]
    # final MA_10* pred_ptc = pred_MA10
    close_MA = pre_MA*(pred_ptc+1)
    # pred_MA_10*10 - pre 9 Close = pred_Close
    pred_close = close_MA*10 - single_df['Close'][-9:].sum()
    return pred_close

In [7]:
model = tf.keras.models.load_model('./model_weight/transformer_btc_multi_factor_v2.0.2.hdf5',
                                   custom_objects={'Time2Vector': Time2Vector, 
                                                   'SingleAttention': SingleAttention,
                                                   'MultiAttention': MultiAttention,
                                                   'TransformerEncoder': TransformerEncoder})

2023-03-19 16:07:30.736385: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 16:07:31.551823: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 44773 MB memory:  -> device: 0, name: NVIDIA Quadro RTX 8000, pci bus id: 0000:21:00.0, compute capability: 7.5


In [2]:
with open('multi_factor_v2.0.2.pickle', 'rb') as input_pickle:
    multi_factor_dic = pickle.load(input_pickle)

zscore_col = multi_factor_dic['zscore'][0]
zscore_std = multi_factor_dic['zscore'][1]
zscore_mean = multi_factor_dic['zscore'][2]
input_col = multi_factor_dic['input_col']

batch_size = 32
seq_len = 128
d_k = 256
d_v = 256
n_heads = 12
ff_dim = 256
valid_percent = 20
test_percent = 10

target_column='Close_MA_ptc_10'
pearson_corr_cutoff = 0.2

In [3]:
df = binance_single_fetch_history_price(coin='BTC/USDT', timeframe='1h', seq_len=seq_len)

In [16]:
timeframes = [10,20,40,60,80,100,120,140,160,180,200]
df_cal = calculate_technical_indicators(df, timeframes=timeframes)
df_zscore, df_zscore_total = calculate_z_score_input_format(df_cal)


In [17]:
df_zscore_total.tail()

,Timestamp,Open,High,Low,Close,Volume,open_MA_10,High_MA_10,Low_MA_10,Close_MA_10,Volume_MA_10,open_MA_ptc_10,High_MA_ptc_10,Low_MA_ptc_10,Close_MA_ptc_10,Volume_MA_ptc_10,Bias,Cmo_10,Atr_10,Cci_10,Volume_pct,ZhangDieFu,ZhenFu,Rsi_10,Ic_10,Macd,Signal,Histogram,UpperBB_10,MiddleBB_10,LowerBB_10,SlowK,SlowD,UpperMAE_10,MiddleMAE_10,LowerMAE_10,OBV,open_MA_20,High_MA_20,Low_MA_20,Close_MA_20,Volume_MA_20,open_MA_ptc_20,High_MA_ptc_20,Low_MA_ptc_20,Close_MA_ptc_20,Volume_MA_ptc_20,Cmo_20,Atr_20,Cci_20,Rsi_20,Ic_20,UpperBB_20,MiddleBB_20,LowerBB_20,UpperMAE_20,MiddleMAE_20,LowerMAE_20,open_MA_40,High_MA_40,Low_MA_40,Close_MA_40,Volume_MA_40,open_MA_ptc_40,High_MA_ptc_40,Low_MA_ptc_40,Close_MA_ptc_40,Volume_MA_ptc_40,Cmo_40,Atr_40,Cci_40,Rsi_40,Ic_40,UpperBB_40,MiddleBB_40,LowerBB_40,UpperMAE_40,MiddleMAE_40,LowerMAE_40,open_MA_60,High_MA_60,Low_MA_60,Close_MA_60,Volume_MA_60,open_MA_ptc_60,High_MA_ptc_60,Low_MA_ptc_60,Close_MA_ptc_60,Volume_MA_ptc_60,Cmo_60,Atr_60,Cci_60,Rsi_60,Ic_60,UpperBB_60,MiddleBB_60,LowerBB_60,UpperMAE_60,MiddleMAE_60,LowerMAE_60,open_MA_80,High_MA_80,Low_MA_80,Close_MA_80,Volume_MA_80,open_MA_ptc_80,High_MA_ptc_80,Low_MA_ptc_80,Close_MA_ptc_80,Volume_MA_ptc_80,Cmo_80,Atr_80,Cci_80,Rsi_80,Ic_80,UpperBB_80,MiddleBB_80,LowerBB_80,UpperMAE_80,MiddleMAE_80,LowerMAE_80,open_MA_100,High_MA_100,Low_MA_100,Close_MA_100,Volume_MA_100,open_MA_ptc_100,High_MA_ptc_100,Low_MA_ptc_100,Close_MA_ptc_100,Volume_MA_ptc_100,Cmo_100,Atr_100,Cci_100,Rsi_100,Ic_100,UpperBB_100,MiddleBB_100,LowerBB_100,UpperMAE_100,MiddleMAE_100,LowerMAE_100,open_MA_120,High_MA_120,Low_MA_120,Close_MA_120,Volume_MA_120,open_MA_ptc_120,High_MA_ptc_120,Low_MA_ptc_120,Close_MA_ptc_120,Volume_MA_ptc_120,Cmo_120,Atr_120,Cci_120,Rsi_120,Ic_120,UpperBB_120,MiddleBB_120,LowerBB_120,UpperMAE_120,MiddleMAE_120,LowerMAE_120,open_MA_140,High_MA_140,Low_MA_140,Close_MA_140,Volume_MA_140,open_MA_ptc_140,High_MA_ptc_140,Low_MA_ptc_140,Close_MA_ptc_140,Volume_MA_ptc_140,Cmo_140,Atr_140,Cci_140,Rsi_140,Ic_140,UpperBB_140,MiddleBB_140,LowerBB_140,UpperMAE_140,MiddleMAE_140,LowerMAE_140,open_MA_160,High_MA_160,Low_MA_160,Close_MA_160,Volume_MA_160,open_MA_ptc_160,High_MA_ptc_160,Low_MA_ptc_160,Close_MA_ptc_160,Volume_MA_ptc_160,Cmo_160,Atr_160,Cci_160,Rsi_160,Ic_160,UpperBB_160,MiddleBB_160,LowerBB_160,UpperMAE_160,MiddleMAE_160,LowerMAE_160,open_MA_180,High_MA_180,Low_MA_180,Close_MA_180,Volume_MA_180,open_MA_ptc_180,High_MA_ptc_180,Low_MA_ptc_180,Close_MA_ptc_180,Volume_MA_ptc_180,Cmo_180,Atr_180,Cci_180,Rsi_180,Ic_180,UpperBB_180,MiddleBB_180,LowerBB_180,UpperMAE_180,MiddleMAE_180,LowerMAE_180,open_MA_200,High_MA_200,Low_MA_200,Close_MA_200,Volume_MA_200,open_MA_ptc_200,High_MA_ptc_200,Low_MA_ptc_200,Close_MA_ptc_200,Volume_MA_ptc_200,Cmo_200,Atr_200,Cci_200,Rsi_200,Ic_200,UpperBB_200,MiddleBB_200,LowerBB_200,UpperMAE_200,MiddleMAE_200,LowerMAE_200
323,2023-03-19-09:00,26971.8,27039.9,26906.8,27039.9,12033.661,27043.19,27144.90,26966.04,27056.46,15086.5908,-0.000496,-0.128920,0.062989,0.084052,-0.154797,11.986812,-7.841736,228.638077,-66.666667,-0.450518,0.822428,1.694582,46.079132,-0.000354,-17.334894,23.758141,-41.093036,27215.700678,27056.46,26897.219322,41.516254,28.268197,27215.700678,27056.46,26897.219322,1481481.059,27163.445,27273.355,27002.065,27145.215,26549.71430,-0.441539,-0.415467,-0.382779,-0.337326,-0.021926,3.848406,271.219260,-66.666667,51.924203,-0.002075,27480.417370,27145.215,26810.012630,27480.417370,27145.215,26810.012630,27204.1075,27343.2175,27039.2550,27218.0575,31317.916300,0.101524,0.029680,0.084098,0.091792,-0.022546,14.225047,301.315375,-66.666667,57.112524,0.003163,27684.854768,27218.0575,26751.260232,27684.854768,27218.0575,26751.260232,26775.951667,26943.311667,26625.533333,26811.620000,37742.634033,0.423609,0.341889,0.420963,0.388803,-0.007521,17.161898,307.715911,66.666667,58.580949,0.013573,28187.868142,26811.620000,25435.371858,28187.868142,26811.620000,25435.371858,26242.96250,26406.37875,26100.43500,26276.84875,36534.199862,

In [18]:
X_test = preprocess_single_sequence_data(df_zscore, seq_len=seq_len)

In [19]:
pred_test = model.predict(X_test)

In [20]:
restore_single_zscore_close_price_v3(pred_test, df_zscore_total)

27273.826829576486

In [20]:
import time
from apscheduler.schedulers.blocking import BlockingScheduler
from datetime import datetime
def job1():
    print(f'工作１啟動: 目前時間{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

def job2():
    print(f'工作２啟動: 目前時間{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
    
def job3():
    print(f'工作３啟動: 目前時間{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

def job4():
    print(f'工作４啟動: 目前時間{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

# 指定時區（一定要指定，否則會失敗）
scheduler = BlockingScheduler(timezone="Asia/Shanghai")

# 每１分鐘執行job1函式
scheduler.add_job(job1, 'interval', minutes=1)

# 每５秒執行job2函式
scheduler.add_job(job2, 'interval', seconds=5)

# 每１秒執行job3函式
scheduler.add_job(job3, 'interval', seconds=1)

# 每週二到日的下午6點30分執行job4函式
scheduler.add_job(job4, 'cron', day_of_week='1-6', hour=18, minute=30)

scheduler.start()

print('Schedule started ...')  # 這行不會被執行

工作３啟動: 目前時間2023-03-19 11:17:09
工作３啟動: 目前時間2023-03-19 11:17:10
工作３啟動: 目前時間2023-03-19 11:17:11
工作３啟動: 目前時間2023-03-19 11:17:12
工作２啟動: 目前時間2023-03-19 11:17:13工作３啟動: 目前時間2023-03-19 11:17:13

工作３啟動: 目前時間2023-03-19 11:17:14
工作３啟動: 目前時間2023-03-19 11:17:15
工作３啟動: 目前時間2023-03-19 11:17:16
工作３啟動: 目前時間2023-03-19 11:17:17
工作２啟動: 目前時間2023-03-19 11:17:18工作３啟動: 目前時間2023-03-19 11:17:18

工作３啟動: 目前時間2023-03-19 11:17:19


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7f48a5a56310>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/weakref.py", line 345, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [2]:
import schedule
import time

def task():
    print("Job Executing!")

# for every n minutes
schedule.every(0.1).minutes.do(task)

while True:
    schedule.run_pending()

Job Executing!
Job Executing!
Job Executing!



KeyboardInterrupt



In [22]:
!pip install schedule

In [28]:
import datetime

now = datetime.datetime.now()
next_hour = now.replace(hour=now.hour+1, minute=0, second=0, microsecond=0)

delta = next_hour - now
seconds_left = delta.seconds
print("距離下一個小時整點還剩下", seconds_left+1, "秒")

距離下一個小時整點還剩下 1554 秒


In [27]:
datetime.datetime.now()

datetime.datetime(2023, 3, 19, 11, 31, 42, 501390)

In [76]:
def predict_single_next_price():
    df = binance_single_fetch_history_price(coin='BTC/USDT', timeframe='1h', seq_len=seq_len)
    timeframes = [10,20,40,60,80,100,120,140,160,180,200]
    df_cal = calculate_technical_indicators(df, timeframes=timeframes)
    df_zscore, df_zscore_total = calculate_z_score_input_format(df_cal)
    X_test = preprocess_single_sequence_data(df_zscore, seq_len=seq_len)
    pred_test = model.predict(X_test)
    pred_close = restore_single_zscore_close_price_v3(pred_test, df_zscore_total)
    output_df = df_zscore_total.loc[327, df_zscore_total.columns[:6]]
    output_df['next_pred_Close'] = pred_close
    output_df = pd.DataFrame(output_df).T
    return pred_close, output_df

In [90]:
pred_close, test_df = predict_single_next_price()

In [75]:
def save_last_row_to_excel(df, filepath):
    # 取得DataFrame最後一筆資料
    last_row = df.iloc[-1,:]
    # 取得前六個columns的內容
    content = last_row[:6]
    # 建立一個新的DataFrame來儲存內容
    df_to_save = pd.DataFrame(content).T
    # 儲存DataFrame成Excel檔
    df_to_save.to_excel(filepath, index=False)

In [92]:
test_df.to_excel('./prediction_log/predcition_log.xlsx', index=False)

In [93]:
aa = pd.read_excel('./prediction_log/predcition_log.xlsx')

In [84]:
# aa = aa.append(test_df, ignore_index=True)

In [16]:
def predict_single_next_price():
    df = binance_single_fetch_history_price(coin='BTC/USDT', timeframe='1h', seq_len=seq_len)
    timeframes = [10,20,40,60,80,100,120,140,160,180,200]
    df_cal = calculate_technical_indicators(df, timeframes=timeframes)
    df_zscore, df_zscore_total = calculate_z_score_input_format(df_cal)
    X_test = preprocess_single_sequence_data(df_zscore, seq_len=seq_len)
    pred_test = model.predict(X_test)
    pred_close = restore_single_zscore_close_price_v3(pred_test, df_zscore_total)
    output_df = df_zscore_total.loc[327, df_zscore_total.columns[:6]]
    output_df['next_pred_Close'] = pred_close
    save_df = pd.DataFrame(output_df).T
    log_file = pd.read_excel('./prediction_log/predcition_log.xlsx')
    log_file = log_file.append(save_df, ignore_index=True)
    log_file.to_excel('./prediction_log/predcition_log.xlsx')
    print('----------')
    print('TimeStamp: ', save_df['Timestamp'][327])
    print('Close: ', save_df['Close'][327])
    print('Next Predict Close: ', save_df['next_pred_Close'][327])
    return pred_close, save_df

In [17]:
pred_close, save_df = predict_single_next_price()

----------
TimeStamp:  2023-03-19-15:00
Close:  27534.3
Next Predict Close:  27488.64407408543


In [12]:
df['Close'][df.index[-1]]

28303.2